<a href="https://colab.research.google.com/github/zatyhan/spam-text-classification/blob/main/fccsmstextclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
  # !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
# !pip install tensorflow-datasets
# import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


# print(tf.__version__)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2024-02-20 21:39:43--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.2’

train-data.tsv.2    100%[===================>] 349.84K  --.-KB/s    in 0.03s   

2024-02-20 21:39:43 (10.4 MB/s) - ‘train-data.tsv.2’ saved [358233/358233]

--2024-02-20 21:39:43--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.2’

valid-data.tsv.2    100%[========

In [64]:
train_data= pd.read_csv(train_file_path, delimiter='\t', names= ['class', 'text'])

In [65]:
train_labels= train_data['class']
train_data= train_data.drop(columns='class')

In [66]:
train_labels= np.array([0 if k =="ham" else 1 for k in train_labels], dtype=np.float32)
train_labels

array([0., 0., 0., ..., 1., 1., 0.], dtype=float32)

In [7]:
train_data

,text
0,ahhhh...just woken up!had a bad dream about u ...
1,you can never do nothing
2,"now u sound like manky scouse boy steve,like! ..."
3,mum say we wan to go then go... then she can s...
4,never y lei... i v lazy... got wat? dat day ü ...
...,...
4174,just woke up. yeesh its late. but i didn't fal...
4175,what do u reckon as need 2 arrange transport i...
4176,free entry into our £250 weekly competition ju...
4177,-pls stop bootydelious (32/f) is inviting you ...


In [47]:
vectorizer= TfidfVectorizer(max_features= 700)
train_transformed= vectorizer.fit_transform(train_data['text']).toarray()

In [52]:
model = tf.keras.Sequential(
    [
        keras.layers.Flatten(input_shape=(train_transformed.shape[1],)),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
    ]
)
model.compile(optimizer='adam', loss= 'binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_9 (Flatten)         (None, 700)               0         
                                                                 
 dense_19 (Dense)            (None, 64)                44864     
                                                                 
 dense_20 (Dense)            (None, 1)                 65        
                                                                 
Total params: 44929 (175.50 KB)
Trainable params: 44929 (175.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [67]:
model.fit(train_transformed, train_labels, batch_size=16, epochs= 100)

Epoch 1/100
262/262 [==============================] - 1s 2ms/step - loss: 2.5388 - accuracy: 0.7475
Epoch 2/100
262/262 [==============================] - 1s 2ms/step - loss: 0.0828 - accuracy: 0.9780
Epoch 3/100
262/262 [==============================] - 1s 3ms/step - loss: 0.0603 - accuracy: 0.9830
Epoch 4/100
262/262 [==============================] - 1s 3ms/step - loss: 0.0486 - accuracy: 0.9871
Epoch 5/100
262/262 [==============================] - 1s 2ms/step - loss: 0.0409 - accuracy: 0.9892
Epoch 6/100
262/262 [==============================] - 1s 2ms/step - loss: 0.0353 - accuracy: 0.9897
Epoch 7/100
262/262 [==============================] - 1s 2ms/step - loss: 0.0308 - accuracy: 0.9909
Epoch 8/100
262/262 [==============================] - 1s 2ms/step - loss: 0.0271 - accuracy: 0.9923
Epoch 9/100
262/262 [==============================] - 1s 2ms/step - loss: 0.0240 - accuracy: 0.9938
Epoch 10/100
262/262 [==============================] - 1s 2ms/step - loss: 0.0213 - accura

In [76]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):

  pred=model.predict(vectorizer.transform([pred_text]).toarray())

  if round(pred[0][0])==1:
    prediction="spam"
  else:
    prediction="ham"

  return (pred_text, prediction)

In [ ]:
predict_message("how are you doing today")

In [77]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 59ms/step
ham
1/1 [==============================] - 0s 22ms/step
spam
1/1 [==============================] - 0s 20ms/step
ham
1/1 [==============================] - 0s 29ms/step
spam
1/1 [==============================] - 0s 37ms/step
spam
1/1 [==============================] - 0s 125ms/step
ham
1/1 [==============================] - 0s 66ms/step
ham
You passed the challenge. Great job!
